# Practicle 9: Explainable AI with LIME

**Aim** Learn how to generate model explaininations using the *Local Interpretable Model-agnostic Explanations* (__[LIME](https://github.com/marcotcr/lime)__) toolbox. 

We be using the *Cleveland Heart Disease Data Set*, avialiable (__[here](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)__) through the University of California Irvine Machine Learning Repository. A CSV of the data has been posted in Keats. Please download it before the lab and place it in the same folder you are running this program from.

The dataset contains 14 clinical attributes from 303 indiviuduals. Each sample is a labelled with either presence (value 1) or absence (value 0)of heart disease. The features in the database are:
1. age: age in years 
2. sex: 1 = male; 0 = female
3. cp: chest pain type. 1= typical angina; 2 = atypical angina; 3 = non-anginal pain ; 4 = asymptomatic 
4. trestbps:resting blood pressure (in mm Hg on admission to the hospital) 
5. chol: serum cholestoral in mg/dl 
6. fbs:  fasting blood sugar > 120 mg/dl. 1 = true; 0 = false
7. restecg: resting electrocardiographic results. 0 = normal; 1: having ST-T wave abnormality; 2 = probable or definite left ventricular hypertrophy
8. thalach: maximum heart rate achieved
9. exang: exercise induced angina. 1 = yes; 0 = no
10. oldpeak: ST segment depression induced by exercise relative to rest 
11. slope: the slope of the peak exercise ST segment. 1 = upsloping; 2 = flat; 3 = downsloping 
12. ca: number of major vessels (0-3) colored by flourosopy 
13. thal: Thalassemia. 3 = normal; 6 = fixed defect; 7 = reversable defect 
14. target: Diagnosis of heart disease. Value 0 = < 50% diameter narrowing; Value 1 = 50% diameter narrowing 

## Step 1 Set-Up

Remember to install LIME *before* starting the tutorial: 

In [ ]:
pip install lime

Import all toolboxes need to support the tutorial

In [ ]:
# Import libraries necessary for this practical
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# Pretty display for notebooks
%matplotlib inline
import seaborn as sns

from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.layers.experimental import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

import lime
from lime import lime_tabular

**Observing Model training** the following function will be used to obsevere the behaviour of the models after they have been trained. 
It plots the loss and accuracy of model after every training epoch

In [ ]:
def plot_learningCurve(history, epoch):
  # Plot training & validation accuracy values
  epoch_range = range(1, epoch+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

## Step 2 Load data and visualise key characteristics

Load the data using Pandas

In [ ]:
data = pd.read_csv("heart.csv")
data.describe()

Plot the distubution of the target scores and gender in the dataset

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.countplot(data=data, x=data.target.replace([1, 0], ['Has Disease', 'Healthy']), ax=ax1)
sns.countplot(data=data, x=data.sex.replace([0, 1], ['Female', 'Male']), ax=ax2)

Lets plot the distribution of disease by age

In [ ]:
pd.crosstab(data.age,data.target).plot(kind="bar",figsize=(16, 7))
plt.title('Heart Disease Frequency grouped by age')
plt.xlabel('Age')
plt.ylabel('Disease Frequency')
plt.legend(["Have Disease", "Healthy"])

Then, let's plot the distribution of disease by gender

In [ ]:
pd.crosstab(data.sex.replace([0, 1], ['Female', 'Male']),data.target).plot(kind="bar",figsize=(10, 5))
plt.xticks(rotation=0)
plt.title('Heart Disease Frequency grouped by Gender')
plt.xlabel('Sex')
plt.ylabel('Desiase Frequency')
plt.legend(["Have Disease", "Healthy"])

Finally, plot the distribution of the features using the sns pairplot function
- **Note** This code takes a couple of minutes to run

Using these plots note down some initial hypothesis on what maybe important features in our classifier? 

In [ ]:
sns.pairplot(data, hue='target')

## Step 3 Perform the necessary preprocessing on data

Convert data from to required train and test arrays

In [ ]:
# Seperate features from labes
X = data.drop(columns='target')
y = data['target']

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42, stratify = y)

# Convert to numpy arrays
X_train = X_train.values 
X_test = X_test.values 

y_train = y_train.values 
y_test = y_test.values

# Parameters needed for setting the input shapes in our model
n_features =  X_train.shape[1]

# one hot encode labels for softmax
y_train_vec = to_categorical(y_train, 2)
y_test_vec = to_categorical(y_test, 2)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Step 4 Train Classifier

**Question** Descibe the model being built below

In [ ]:
# define the keras model
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=n_features, activation='relu'))  
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    print(model.summary())
    return model

**Question** What are the key characteristic of our model training set-up?

In [ ]:
model = create_model()

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

EPOCHS = 64
BATCH_SIZE = 32

history = model.fit(
    X_train, y_train_vec, 
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split = 0.2,
    shuffle=True,
    verbose=0
)

plot_learningCurve(history, EPOCHS)

**Question** We are using softmax and categorical_crossentropy in this task as LIME works with probabilistic outputs. If we did not need probabilistic outputs, what other final layer activation function and loss function combination could we use for this task?

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test_vec)
print("test loss, test acc:", results)

y_pred_vec = model.predict(X_test)
y_pred = np.argmax(y_pred_vec, axis=1)

cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True)

To help find interesting example to expalin, we need identify all correctly and incorrectly classified examples per class. The following function identifes all True Positive, False Negative, False Positive and True Negative samples in the test data

**Question** What do these terms (True Positive etc) mean?

In [ ]:
def perf_measure(y_actual, y_pred):
    TP = []
    FP = []
    TN = []
    FN = []
    counter = 0

    for i in range(len(y_pred)): 
        if y_actual[i]==y_pred[i]==1:
           TP.append(counter)
        if y_pred[i]==1 and y_actual[i]!=y_pred[i]:
           FP.append(counter)
        if y_actual[i]==y_pred[i]==0:
           TN.append(counter)
        if y_pred[i]==0 and y_actual[i]!=y_pred[i]:
           FN.append(counter)
        counter += 1

    return(TP, FP, TN, FN)

In [ ]:
True_Positive, False_Negative, False_Positive, True_Negative = perf_measure(y_test, y_pred)

## Step 5 Create Explanations

To start explaining the model we need to create a tabular explainer `object`, which expects the following parameters:
- `training_data`: The data used to train the model. This must be in a Numpy array format.
- `feature_names`: The column names from the training set
- `class_names`: The distinct classes from the target variable
- `mode`: The type of problem being solved (classification in this case)

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_train,
    feature_names=X.columns,
    class_names=['absence','presence'],
    mode='classification'
)

**We can now start explaining!** 

Call the explain_instance function to explain the prediction. 

The following parameters are required:
- `data_row`: a single observation from the dataset
- `predict_fn`: a function used to make predictions

Explaining a correctly classified heart disease instance. 

**Question** Do any of the features used in the explination match your inital hypothises on important features? 

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[True_Positive[0]], 
    predict_fn=model.predict
)

exp.show_in_notebook(show_table=True)

Explaining a incorrectly classified heart disease instance

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[False_Positive[0]], 
    predict_fn=model.predict
)

exp.show_in_notebook(show_table=True)

Explaining a correctly classified absence of heart disease instance

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[True_Negative[0]], 
    predict_fn=model.predict
)

exp.show_in_notebook(show_table=True)

Explaining a incorrectly classified absence of heart disease instance

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[False_Negative[0]], 
    predict_fn=model.predict
)

exp.show_in_notebook(show_table=True)

## Exercise 1

Create LIME explinations for a *different* classifier (prefereably not a slightly different feedforward neural network)

## Exercise 2

Implement LIME explinations for the *Breast Cancer Wisconsin Diagnostic Dataset* from Practical 4